In [3]:
from basebot22.basebot import BaseBot
from datetime import datetime, timedelta
import pandas as pd
bot = BaseBot("testbot",) #  backendurl = "http://tradingbot-baseimage-service:8000"
# put the above array into one line
allowedStocks = ["AAPL", "MSFT", "GOOG", "TSLA", "AMD", "AMZN", "DG", "KDP", "LLY", "NOC", "NVDA", "PGR", "TEAM", "UNH", "WM", "URTH", "IWDA.AS", "EEM", "XAIX.DE", "BTEC.L", "L0CK.DE", "2B76.DE", "W1TA.DE", "RENW.DE", "BNXG.DE", "BTC-USD", "ETH-USD", "AVAX-USD", "GLD", "TMF", "FAS", "TQQQ", "QQQ", "UUP", "META", "PYPL", "ADBE", "UPRO", "BSV", "SQQQ", "NTSX", "DBMF", "VDE", "VNQ", "VHT", "VFH", "VOX", "VPU", "VAW", "VGT", "VIS", "VDC", "VCR", "VLUE", "FNDX", "VTV", "RWL", "DBA", "SHV", "DBB", "DBO", "URA", "WOOD", "DBE"]

In [33]:
datas = dict()
datas_train = dict()
datas_test = dict()
for ticker in allowedStocks:
    datas[ticker] = bot.getData(ticker, start_date = datetime.now() - timedelta(days=int(365*4)))
    datas_train[ticker] = datas[ticker].iloc[:int(len(datas[ticker])*0.7)]
    datas_test[ticker] = datas[ticker].iloc[int(len(datas[ticker])*0.7):]

In [16]:
from tqdm import tqdm
smallsmas = [5, 10, 20, 50]
bigsmas = [100, 200, 500, 1000]

startMoney = 10000
bestWin = -99999999
bestSettings = dict()
for smalls in tqdm(smallsmas):
    for bigs in bigsmas:
        stocks = dict(USD = startMoney)
        track = []
        for ticker in allowedStocks:
            datas_test[ticker]["smallSma"] = datas_test[ticker].adj_close.rolling(smalls).mean()
            datas_train[ticker]["bigSma"] = datas_train[ticker].adj_close.rolling(bigs).mean()
        for idate in range(len(datas_train["AAPL"])):
            toBuy = []
            for ticker in allowedStocks:
                holdingNr = stocks.get(ticker, 0)
                if idate > len(datas_train[ticker]) - 1:
                    # different lenghts and sheeet
                    # sell it here and now
                    if holdingNr > 0:
                        print("sell", ticker, "on", idate)
                        stocks["USD"] += holdingNr * datas_train[ticker].iloc[-1]["adj_close"]
                        stocks[ticker] = 0
                    continue
                if datas_train[ticker].iloc[idate]["smallSma"] > datas_train[ticker].iloc[idate]["bigSma"] and holdingNr == 0:
                    # buy
                    toBuy.append(ticker) # buy later bc we first need cash
                elif datas_train[ticker].iloc[idate]["smallSma"] < datas_train[ticker].iloc[idate]["bigSma"] and holdingNr > 0:
                    # sell
                    print("sell", ticker, "on", idate)
                    stocks["USD"] += holdingNr * datas_train[ticker].iloc[idate]["adj_close"]
                    stocks[ticker] = 0
            if len(toBuy) > 0:
                investPerStock = stocks["USD"] / len(toBuy)
                if investPerStock > 50:
                    for ticker in toBuy:
                        print("buy", ticker, "on", idate)
                        howMany = investPerStock / datas_train[ticker].iloc[idate]["adj_close"]
                        cost = howMany * datas_train[ticker].iloc[idate]["adj_close"]
                        if cost > stocks["USD"]:
                            print("not enough money to buy", ticker, "on", idate)
                            continue
                        stocks[ticker] = howMany
                        stocks["USD"] -= cost
            track.append(stocks["USD"] + sum([stocks.get(ticker,0) * datas_train[ticker].iloc[idate]["adj_close"] for ticker in stocks.keys() if ticker != "USD" and stocks.get(ticker,0) > 0]))
        print("smallSma", smalls, "bigSma", bigs, "endMoney", track[-1])
        if track[-1] > bestWin:
            bestWin = track[-1]
            bestSettings = dict(smalls = smalls, bigs = bigs, bestWin = bestWin, portfolio = track)
print("bestSettings", bestSettings)

  0%|          | 0/4 [00:00<?, ?it/s]/tmp/ipykernel_606/1391873527.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas_train[ticker]["smallSma"] = datas_train[ticker].adj_close.rolling(smalls).mean()
/tmp/ipykernel_606/1391873527.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas_train[ticker]["bigSma"] = datas_train[ticker].adj_close.rolling(bigs).mean()


buy MSFT on 99
buy AMD on 99
buy AMZN on 99
buy DG on 99
buy KDP on 99
buy NOC on 99
buy PGR on 99
buy TEAM on 99
buy WM on 99
buy URTH on 99
buy IWDA.AS on 99
buy L0CK.DE on 99
buy W1TA.DE on 99
buy BNXG.DE on 99
buy BTC-USD on 99
buy ETH-USD on 99
buy TMF on 99
buy FAS on 99
buy QQQ on 99
buy UUP on 99
buy META on 99
buy PYPL on 99
buy ADBE on 99
buy BSV on 99
buy NTSX on 99
buy DBMF on 99
buy VNQ on 99
buy VFH on 99
buy VOX on 99
buy VPU on 99
buy VGT on 99
buy VDC on 99
buy SHV on 99
not enough money to buy SHV on 99
sell URTH on 100
sell QQQ on 100
buy SHV on 100
sell VFH on 101
buy SQQQ on 101
buy DBA on 101
sell VOX on 102
sell VGT on 102
buy XAIX.DE on 102
buy GLD on 102
sell AMZN on 103
sell L0CK.DE on 103
buy VFH on 103
buy VGT on 103
sell META on 104
sell SQQQ on 105
sell DBA on 105
buy URTH on 105
buy RENW.DE on 105
buy VAW on 105
buy VIS on 105
buy VTV on 105
buy RWL on 105
not enough money to buy RWL on 105
sell RENW.DE on 106
sell UUP on 118
sell XAIX.DE on 126
sell KDP 

 25%|██▌       | 1/4 [00:44<02:14, 44.92s/it]

smallSma 5 bigSma 1000 endMoney 10000
buy AAPL on 99
buy MSFT on 99
buy AMD on 99
buy AMZN on 99
buy DG on 99
buy KDP on 99
buy NOC on 99
buy PGR on 99
buy TEAM on 99
buy WM on 99
buy URTH on 99
buy IWDA.AS on 99
buy L0CK.DE on 99
buy W1TA.DE on 99
buy BNXG.DE on 99
buy BTC-USD on 99
buy ETH-USD on 99
buy TMF on 99
buy FAS on 99
buy TQQQ on 99
buy QQQ on 99
buy UUP on 99
buy META on 99
buy PYPL on 99
buy ADBE on 99
buy UPRO on 99
buy BSV on 99
buy NTSX on 99
buy DBMF on 99
buy VNQ on 99
buy VFH on 99
buy VOX on 99
buy VPU on 99
buy VGT on 99
buy VIS on 99
buy VDC on 99
buy VCR on 99
buy FNDX on 99
buy VTV on 99
buy RWL on 99
buy SHV on 99
buy DBO on 99
buy DBE on 99
sell AAPL on 100
sell VIS on 100
sell FNDX on 100
sell RWL on 100
sell DBO on 100
buy XAIX.DE on 100
sell TQQQ on 101
sell UPRO on 101
sell VCR on 101
sell VTV on 101
sell DBE on 101
sell QQQ on 103
sell URTH on 104
buy GLD on 104
sell VOX on 105
buy URTH on 105
sell L0CK.DE on 106
buy DBA on 106
sell META on 109
sell UUP o

 50%|█████     | 2/4 [01:29<01:29, 44.88s/it]

smallSma 10 bigSma 1000 endMoney 10000
buy AAPL on 99
buy MSFT on 99
buy AMD on 99
buy AMZN on 99
buy DG on 99
buy KDP on 99
buy NOC on 99
buy PGR on 99
buy TEAM on 99
buy WM on 99
buy URTH on 99
buy IWDA.AS on 99
buy L0CK.DE on 99
buy 2B76.DE on 99
buy W1TA.DE on 99
buy BNXG.DE on 99
buy BTC-USD on 99
buy ETH-USD on 99
buy TMF on 99
buy FAS on 99
buy TQQQ on 99
buy QQQ on 99
buy UUP on 99
buy META on 99
buy PYPL on 99
buy ADBE on 99
buy UPRO on 99
buy BSV on 99
buy NTSX on 99
buy DBMF on 99
buy VNQ on 99
buy VFH on 99
buy VOX on 99
buy VPU on 99
buy VGT on 99
buy VIS on 99
buy VDC on 99
buy VCR on 99
buy FNDX on 99
buy VTV on 99
buy RWL on 99
buy SHV on 99
buy DBO on 99
buy DBE on 99
not enough money to buy DBE on 99
buy DBE on 100
sell 2B76.DE on 102
sell RWL on 103
sell DBO on 103
sell DBE on 104
sell FNDX on 105
sell TQQQ on 106
sell VIS on 106
buy XAIX.DE on 106
buy GLD on 106
sell AAPL on 110
buy AVAX-USD on 112
buy DBA on 112
sell XAIX.DE on 134
buy AAPL on 134
buy UNH on 134
bu

 75%|███████▌  | 3/4 [02:14<00:44, 44.76s/it]

smallSma 20 bigSma 1000 endMoney 10000
buy AAPL on 99
buy MSFT on 99
buy GOOG on 99
buy AMD on 99
buy AMZN on 99
buy DG on 99
buy KDP on 99
buy LLY on 99
buy NOC on 99
buy NVDA on 99
buy PGR on 99
buy TEAM on 99
buy WM on 99
buy URTH on 99
buy IWDA.AS on 99
buy EEM on 99
buy XAIX.DE on 99
buy L0CK.DE on 99
buy 2B76.DE on 99
buy W1TA.DE on 99
buy RENW.DE on 99
buy BNXG.DE on 99
buy BTC-USD on 99
buy ETH-USD on 99
buy TMF on 99
buy FAS on 99
buy TQQQ on 99
buy QQQ on 99
buy UUP on 99
buy META on 99
buy PYPL on 99
buy ADBE on 99
buy UPRO on 99
buy BSV on 99
buy NTSX on 99
buy DBMF on 99
buy VDE on 99
buy VNQ on 99
buy VFH on 99
buy VOX on 99
buy VPU on 99
buy VAW on 99
buy VGT on 99
buy VIS on 99
buy VDC on 99
buy VCR on 99
buy VLUE on 99
buy FNDX on 99
buy VTV on 99
buy RWL on 99
buy SHV on 99
buy DBB on 99
buy DBO on 99
buy DBE on 99
sell LLY on 100
sell DBB on 102
sell RENW.DE on 103
sell XAIX.DE on 104
sell VDE on 106
sell EEM on 108
sell VLUE on 113
sell GOOG on 115
sell NVDA on 115


100%|██████████| 4/4 [02:58<00:00, 44.67s/it]

smallSma 50 bigSma 1000 endMoney 10000
bestSettings {'smalls': 50, 'bigs': 100, 'bestWin': 26288.248252008663, 'portfolio': [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000.0, 9886.17207447888, 9840.602297428677, 10012.776177841588, 10078.138741499604, 10157.280015342476, 10258.912038710272, 10314.75198835043, 10320.361046750273, 10296.44736587029

the best result of finding one SMA value for every stock is: 

{'smalls': 50, 'bigs': 100, 'bestWin': 26288.248252008663,

the prob with that is, that it could be one stock that has super high earnings, and the others are just poor

we did a test and this performed worse than in the train

next: try out one sma per stock

## start over bc something is off

In [75]:
import numpy as np
def oneRun(data):
    df, smallSMA, bigSMA, ticker = data
    # print(f"starting at {df.index[0]}, ending at {df.index[-1]}")
    money = 10000
    stocks = 0
    nrTrades = 0
    df.loc[:,"smallSMA"] = df.adj_close.rolling(smallSMA).mean()
    df.loc[:,"bigSMA"] = df.adj_close.rolling(bigSMA).mean()
    previousSmall = df["smallSMA"].shift(1)
    previousBig = df["bigSMA"].shift(1)
    upcross = df.index[(df["smallSMA"] > df["bigSMA"]) & (previousSmall < previousBig)]
    downcross = df.index[(df["smallSMA"] < df["bigSMA"]) & (previousSmall > previousBig)]
    upcross = [(idx,"up") for idx in upcross]
    downcross = [(idx,"down") for idx in downcross]
    merged = sorted(upcross + downcross)
    boughtAt = 0
    tradeWins = []
    for date, direction in merged:
        if direction == "up" and stocks == 0:
            # buy
            stocks += money / df.loc[date,"adj_close"]
            money = 0
            boughtAt = df.loc[date,"adj_close"]
        elif direction == "down" and stocks > 0:
            # sell
            money += stocks * df.loc[date,"adj_close"]
            stocks = 0
            nrTrades += 1
            tradeWins.append(df.loc[date,"adj_close"] / boughtAt - 1) # is the pct how much we won or lost
    # last day sell
    money += stocks * df.iloc[-1]["adj_close"]
    return ticker, money, nrTrades, np.median(tradeWins), smallSMA, bigSMA
    
ticker, money, nrTrades, tradeWins, smallSMA, bigSMA = oneRun([datas_train["AAPL"], 20, 100, "AAPL"])
win

21753.70907014834

In [76]:
import warnings
from random import randint
from multiprocessing import Pool
warnings.filterwarnings("ignore")
import time

SMALL = list(range(60,200,10)) + [randint(1,500) for _ in range(50)]
BIG = list(range(200,1000,50)) +  [randint(1,5000) for _ in range(50)]

best = dict()
possibilities = []
for ticker in allowedStocks:
    for smallSMA in SMALL:
        for bigSMA in BIG:
            possibilities.append((datas_train[ticker], smallSMA, bigSMA, ticker))
pool = Pool()
start = time.time()
r = pool.map_async(oneRun, possibilities).get()
took = time.time() - start
print(f"calculating {len(possibilities)} possibilities took {took} seconds, or {took/len(possibilities)} seconds per possibility")
result = pd.DataFrame(r)
result.columns = ["ticker", "money", "nrTrades", "tradeWinsMedian", "smallSMA", "bigSMA"]

    #         win, nrTrades = oneRun(datas_train[ticker], smallSMA, bigSMA)
    #         if ticker not in best:
    #             best[ticker] = {"win":win, "smallSMA":smallSMA, "bigSMA":bigSMA, "nrTrades":nrTrades}
    #         elif win > best[ticker]["win"]:
    #             best[ticker] = {"win":win, "smallSMA":smallSMA, "bigSMA":bigSMA, "nrTrades":nrTrades}
    # print(f"the best for {ticker} is {best[ticker]}")

calculating 270336 possibilities took 56.09948372840881 seconds, or 0.00020751762150956148 seconds per possibility


In [77]:
result = result.sort_values("tradeWinsMedian", ascending=False)
result

,ticker,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
13068,TSLA,77451.882972,1,9.945172,120,200
15510,TSLA,76706.284900,1,9.792746,116,200
14190,TSLA,76706.284900,1,9.792746,115,200
15576,TSLA,76706.284900,1,9.792746,116,200
13002,TSLA,85461.784783,1,9.715830,110,200
...,...,...,...,...,...,...
270331,DBE,10000.000000,0,NaN,232,1818
270332,DBE,10000.000000,0,NaN,232,3540
270333,DBE,10000.000000,0,NaN,232,2993
270334,DBE,10000.000000,0,NaN,232,2588


just grab those where we have > 4 trades, then grab the highest win per stock

In [57]:
# res = result[result["nrTrades"] > 2]
# res.head()

,ticker,money,nrTrades,smallSMA,bigSMA
6032,ETH-USD,120640.388668,4,190,200
6016,ETH-USD,117839.577135,3,180,200
5792,BTC-USD,58080.558591,3,180,200
5808,BTC-USD,56149.298867,5,190,200
5729,BTC-USD,55221.078751,3,140,250


In [78]:
# grab the highest money for each ticker
best = result.groupby("ticker").first()
# best.loc["ETH-USD"]
best = best.sort_values("money", ascending=False)
best

,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
ticker,,,,,
ETH-USD,336346.099406,2,8.320259,115,156
TSLA,77451.882972,1,9.945172,120,200
BTC-USD,66744.450282,1,5.674445,188,300
AVAX-USD,64699.332001,1,5.360121,180,156
NVDA,61611.773956,1,5.161177,70,156
...,...,...,...,...,...
NTSX,10551.884551,1,0.055188,190,156
UUP,10214.699306,1,0.021470,482,502
SHV,10001.811161,1,0.000181,431,2


## do a test run using datas_test and the above combinations, equal weights

In [82]:
testResults = []
for ticker, (money, nrTrades, tradeWinsMedian, smallSMA, bigSMA) in best.iterrows():
    _, money, nrTrades,tradeWinsMedian, _, _ = oneRun([datas_test[ticker], int(smallSMA), int(bigSMA), ticker])
    testResults.append((ticker, money, nrTrades, tradeWinsMedian, smallSMA, bigSMA))
testResults = pd.DataFrame(testResults)
testResults.columns = ["ticker", "money", "nrTrades", "tradeWinsMedian", "smallSMA", "bigSMA"]
testResults = testResults.sort_values("money", ascending=False)
testResults.head()

,ticker,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
63,SQQQ,15012.979288,4,0.109396,115.0,2.0
52,DBB,10930.968759,0,NaN,214.0,200.0
34,PGR,10469.972770,0,NaN,175.0,200.0
44,VPU,10279.939735,0,NaN,190.0,156.0
43,DBA,10132.181102,0,NaN,232.0,200.0


first result looks like shite as expected!
adapt best to give better results

In [86]:
# only take where more than 4 trades
best = result[result["nrTrades"] > 1]
best = best.groupby("ticker").first()
# best.loc["ETH-USD"]
best = best.sort_values("money", ascending=False)
best

,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
ticker,,,,,
ETH-USD,336346.099406,2,8.320259,115,156
TSLA,157540.369561,2,5.744201,25,156
AVAX-USD,81371.167485,2,2.983039,175,156
BTC-USD,55221.078751,2,2.221620,140,250
NVDA,41694.100308,2,1.153823,19,156
...,...,...,...,...,...
BSV,10141.702252,4,0.002466,130,2
SHV,9998.188068,2,0.000136,224,156
DBO,8778.432047,3,0.115431,160,156


In [87]:
testResults = []
for ticker, (money, nrTrades,tradeWinsMedian, smallSMA, bigSMA) in best.iterrows():
    _, money, nrTrades, tradeWinsMedian, _, _ = oneRun([datas_test[ticker], int(smallSMA), int(bigSMA), ticker])
    testResults.append((ticker, money, nrTrades, tradeWinsMedian, smallSMA, bigSMA))
testResults = pd.DataFrame(testResults)
testResults.columns = ["ticker", "money", "nrTrades", "tradeWinsMedian", "smallSMA", "bigSMA"]
testResults = testResults.sort_values("money", ascending=False)
testResults.head()

,ticker,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
63,SQQQ,14280.515953,3,0.132297,190.0,2.0
41,UNH,11055.433410,5,0.016251,188.0,2.0
17,VDE,10732.622966,1,0.073262,150.0,156.0
15,LLY,10717.961941,1,-0.038000,40.0,156.0
49,VPU,10595.329116,2,0.030524,170.0,156.0


### we want to maximize the average tradeWinsMedian for all stocks

In [96]:
## use train data for this shiet

for minTrades in [0,1,2,3,4,5,6,7,8,9,10]:
    best = result[result["nrTrades"] > minTrades]
    best = best.sort_values("tradeWinsMedian", ascending=False)
    best = best.groupby("ticker").first()
    # best.loc["ETH-USD"]
    best = best.sort_values("money", ascending=False)
    
    # simulate
    testResults = []
    for ticker, (money, nrTrades,tradeWinsMedian, smallSMA, bigSMA) in best.iterrows():
        _, money, nrTrades, tradeWinsMedian, _, _ = oneRun([datas_train[ticker], int(smallSMA), int(bigSMA), ticker])
        testResults.append((ticker, money, nrTrades, tradeWinsMedian, smallSMA, bigSMA))
    testResults = pd.DataFrame(testResults)
    testResults.columns = ["ticker", "money", "nrTrades", "tradeWinsMedian", "smallSMA", "bigSMA"]
    testResults = testResults.sort_values("money", ascending=False)
    globalAvgTradesWinsMed = testResults["tradeWinsMedian"].median()
    print(f"minTrades: {minTrades}, globalAvgTradesWinsMed: {globalAvgTradesWinsMed}, avgWin: {testResults['money'].mean()}, best: {testResults.iloc[0]['money']}, worst: {testResults.iloc[-1]['money']}")

minTrades: 0, globalAvgTradesWinsMed: 0.3923417131563056, avgWin: 23805.986681585473, best: 336346.09940609505, worst: 819.0344532796452
minTrades: 1, globalAvgTradesWinsMed: 0.18074487081710666, avgWin: 24158.804277231073, best: 336346.09940609505, worst: 770.3116873688459
minTrades: 2, globalAvgTradesWinsMed: 0.07839924603406667, avgWin: 16818.43572583094, best: 133193.00288056472, worst: 1602.5818087626835
minTrades: 3, globalAvgTradesWinsMed: 0.0513493093837587, avgWin: 13863.195372932882, best: 39020.67212989732, worst: 1602.5818087626835
minTrades: 4, globalAvgTradesWinsMed: 0.04524316763497532, avgWin: 15089.985401789309, best: 133043.48062859068, worst: 1602.5818087626835
minTrades: 5, globalAvgTradesWinsMed: 0.03520169544165147, avgWin: 13148.718952563802, best: 40212.67539091112, worst: 1602.5818087626835
minTrades: 6, globalAvgTradesWinsMed: 0.03112403241787598, avgWin: 12986.08976608875, best: 26755.570105130813, worst: 1602.5818087626835
minTrades: 7, globalAvgTradesWinsMe

## manually selecting 
minTrades: 4, globalAvgTradesWinsMed: 0.04524316763497532, avgWin: 15089.985401789309, best: 133043.48062859068, worst: 1602.5818087626835


In [112]:
## next only keep stocks that have at least median win of >0
best = result[result["nrTrades"] > 4]
best = best[best["tradeWinsMedian"] > 0.]
best = best.sort_values("tradeWinsMedian", ascending=False)
best = best.groupby("ticker").first()
best = best.sort_values("tradeWinsMedian", ascending=False)
best

,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
ticker,,,,,
BTC-USD,38864.967548,5,0.291532,152,156
AVAX-USD,24093.817424,5,0.239281,175,2
ETH-USD,133043.480629,5,0.204796,150,156
TSLA,17971.437878,6,0.107978,170,2
UNH,21643.042691,5,0.097217,153,156
...,...,...,...,...,...
DBB,9237.174452,8,0.019881,60,2
DBA,9536.781305,11,0.016369,72,2
UUP,9756.264673,5,0.006029,300,2


## test it


In [113]:
testResults = []
for ticker, (money, nrTrades,tradeWinsMedian, smallSMA, bigSMA) in best.iterrows():
    _, money, nrTrades, tradeWinsMedian, _, _ = oneRun([datas_test[ticker], int(smallSMA), int(bigSMA), ticker])
    testResults.append((ticker, money, nrTrades, tradeWinsMedian, smallSMA, bigSMA))
testResults = pd.DataFrame(testResults)
testResults.columns = ["ticker", "money", "nrTrades", "tradeWinsMedian", "smallSMA", "bigSMA"]
testResults = testResults.sort_values("money", ascending=False)
testResults.head()

,ticker,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
56,SQQQ,16234.832582,8,0.074580,60.0,2.0
49,NOC,12959.977738,6,0.039571,140.0,2.0
42,VDE,12007.870433,2,0.096263,170.0,2.0
29,URA,11801.205398,6,0.022539,130.0,2.0
31,LLY,11697.151574,5,0.024343,120.0,2.0


In [114]:
testResults["money"].mean()

9871.99345620604

### next introduce weights

In [117]:
best["pctGain"] = best["money"] / 10000
print(best)
weights = best["money"] / best["money"].sum()
weights

                  money  nrTrades  tradeWinsMedian  smallSMA  bigSMA  \
ticker                                                                 
BTC-USD    38864.967548         5         0.291532       152     156   
AVAX-USD   24093.817424         5         0.239281       175       2   
ETH-USD   133043.480629         5         0.204796       150     156   
TSLA       17971.437878         6         0.107978       170       2   
UNH        21643.042691         5         0.097217       153     156   
...                 ...       ...              ...       ...     ...   
DBA        10101.658316        13         0.010753        60       2   
UUP        10136.466361        17         0.005607        40       2   
VDE        18102.584228         7         0.002210       153     156   
BSV        10324.470198         5         0.001681       160       2   
SHV        10008.141582        10         0.000090       190       2   

            pctGain  
ticker               
BTC-USD    3.886497

ticker
BTC-USD     0.039618
AVAX-USD    0.024560
ETH-USD     0.135620
TSLA        0.018319
UNH         0.022062
              ...   
DBA         0.010297
UUP         0.010333
VDE         0.018453
BSV         0.010524
SHV         0.010202
Name: money, Length: 63, dtype: float64

In [118]:
## train money win with weights
startMoney = 10000
gain = weights * startMoney
gain

ticker
BTC-USD      396.175334
AVAX-USD     245.603606
ETH-USD     1356.196819
TSLA         183.194297
UNH          220.621300
               ...     
DBA          102.972628
UUP          103.327449
VDE          184.531155
BSV          105.243892
SHV          102.019353
Name: money, Length: 63, dtype: float64

### simulate that sheeet with weights



In [120]:
testResults = []
for ticker, (money, nrTrades,tradeWinsMedian, smallSMA, bigSMA, pctGain) in best.iterrows():
    _, money, nrTrades, tradeWinsMedian, _, _ = oneRun([datas_test[ticker], int(smallSMA), int(bigSMA), ticker])
    testResults.append((ticker, money, nrTrades, tradeWinsMedian, smallSMA, bigSMA))
testResults = pd.DataFrame(testResults)
testResults.columns = ["ticker", "money", "nrTrades", "tradeWinsMedian", "smallSMA", "bigSMA"]
testResults = testResults.sort_values("money", ascending=False)
testResults.head()

,ticker,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
48,NOC,12959.977738,6,0.039571,140.0,2.0
28,URA,11801.205398,6,0.022539,130.0,2.0
30,LLY,11697.151574,5,0.024343,120.0,2.0
15,PGR,11664.739993,2,0.034288,150.0,156.0
27,BTEC.L,11603.128530,7,0.018597,130.0,2.0


In [121]:
testResults["money"].mean()

9770.70376259353

In [124]:
testResults["gain"] = testResults["money"] / 10000
testResults = testResults.set_index("ticker")
testResults

,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA,gain
ticker,,,,,,
NOC,12959.977738,6,0.039571,140.0,2.0,1.295998
URA,11801.205398,6,0.022539,130.0,2.0,1.180121
LLY,11697.151574,5,0.024343,120.0,2.0,1.169715
PGR,11664.739993,2,0.034288,150.0,156.0,1.166474
BTEC.L,11603.128530,7,0.018597,130.0,2.0,1.160313
...,...,...,...,...,...,...
FAS,7501.899661,6,0.050469,72.0,2.0,0.750190
AMZN,7025.971623,1,-0.297403,140.0,156.0,0.702597
TEAM,6235.257766,1,0.085724,175.0,2.0,0.623526


In [126]:
## calculate the money if we would have invested with weights
weightedMoney = testResults["money"] * weights
weightedMoney.sum()

9528.006319913537

## well doesnt look too good, but lets create a trading hypothesis from this

we want minimum 4 trades to have occured

we want tradeWinsMedian to be > 0.

sort by tradeWinsMedian

weigh according to that


In [127]:
import warnings
from random import randint
from multiprocessing import Pool
warnings.filterwarnings("ignore")
import time

SMALL = list(range(60,200,10)) + [randint(1,500) for _ in range(50)]
BIG = list(range(200,1000,50)) +  [randint(1,5000) for _ in range(50)]

best = dict()
possibilities = []
for ticker in allowedStocks:
    for smallSMA in SMALL:
        for bigSMA in BIG:
            possibilities.append((datas[ticker], smallSMA, bigSMA, ticker))
pool = Pool()
start = time.time()
r = pool.map_async(oneRun, possibilities).get()
took = time.time() - start
print(f"calculating {len(possibilities)} possibilities took {took} seconds, or {took/len(possibilities)} seconds per possibility")
result = pd.DataFrame(r)
result.columns = ["ticker", "money", "nrTrades", "tradeWinsMedian", "smallSMA", "bigSMA"]

    #         win, nrTrades = oneRun(datas_train[ticker], smallSMA, bigSMA)
    #         if ticker not in best:
    #             best[ticker] = {"win":win, "smallSMA":smallSMA, "bigSMA":bigSMA, "nrTrades":nrTrades}
    #         elif win > best[ticker]["win"]:
    #             best[ticker] = {"win":win, "smallSMA":smallSMA, "bigSMA":bigSMA, "nrTrades":nrTrades}
    # print(f"the best for {ticker} is {best[ticker]}")

calculating 270336 possibilities took 47.272902965545654 seconds, or 0.00017486721326625258 seconds per possibility


In [131]:
## next only keep stocks that have at least median win of >0
best = result[result["nrTrades"] > 4]
best = best[best["tradeWinsMedian"] > 0.]
best = best[best["money"] > 13000.] # dont even care about those small stoxx
best = best.sort_values("tradeWinsMedian", ascending=False)
best = best.groupby("ticker").first()
best = best.sort_values("tradeWinsMedian", ascending=False)
best

,money,nrTrades,tradeWinsMedian,smallSMA,bigSMA
ticker,,,,,
ETH-USD,19618.158857,5,0.355900,140,48
URA,28614.916997,5,0.344667,110,48
NVDA,27804.952313,6,0.283601,35,73
LLY,19694.929556,5,0.210618,174,185
BTC-USD,13770.437698,8,0.194728,79,48
TSLA,139031.679417,8,0.177024,35,73
AMD,16726.238299,7,0.174756,91,73
PGR,17935.478886,5,0.149191,210,217
VDE,15601.871041,5,0.137062,110,73


In [ ]:
best.to_csv("smabest.csv")

In [133]:
# create weights accoridng to money

weights = best["money"] / best["money"].sum()
weights = weights.sort_values(ascending=False)
weights

ticker
TSLA        0.135235
FAS         0.045317
URA         0.027833
NVDA        0.027046
TEAM        0.023537
AVAX-USD    0.021031
DBE         0.020351
DG          0.019983
LLY         0.019157
ETH-USD     0.019082
VAW         0.019012
NOC         0.018216
UPRO        0.018077
PGR         0.017446
WM          0.017246
GOOG        0.017171
VIS         0.016526
UNH         0.016485
AMD         0.016269
WOOD        0.016135
VPU         0.015534
KDP         0.015307
VDE         0.015176
VHT         0.015057
IWDA.AS     0.015051
VDC         0.014592
VFH         0.014535
META        0.014533
VTV         0.014421
L0CK.DE     0.014337
BTEC.L      0.014269
2B76.DE     0.014208
MSFT        0.014005
DBB         0.013856
VNQ         0.013703
FNDX        0.013628
XAIX.DE     0.013582
DBA         0.013530
TQQQ        0.013504
PYPL        0.013400
BTC-USD     0.013394
DBO         0.013394
AMZN        0.013393
RENW.DE     0.013353
AAPL        0.013240
GLD         0.013069
VLUE        0.012992
EEM   

In [138]:
weights.to_csv("smaweights.csv")

In [139]:
diff = weights["TSLA"] - 0.04 # value of FAS
weights["TSLA"] = 0.04
weights += diff / (len(weights) )
weights

ticker
TSLA        0.041764
FAS         0.047080
URA         0.029597
NVDA        0.028809
TEAM        0.025300
AVAX-USD    0.022795
DBE         0.022115
DG          0.021746
LLY         0.020921
ETH-USD     0.020846
VAW         0.020776
NOC         0.019979
UPRO        0.019841
PGR         0.019209
WM          0.019010
GOOG        0.018935
VIS         0.018289
UNH         0.018249
AMD         0.018033
WOOD        0.017899
VPU         0.017297
KDP         0.017070
VDE         0.016939
VHT         0.016821
IWDA.AS     0.016814
VDC         0.016355
VFH         0.016299
META        0.016296
VTV         0.016184
L0CK.DE     0.016101
BTEC.L      0.016033
2B76.DE     0.015971
MSFT        0.015769
DBB         0.015620
VNQ         0.015466
FNDX        0.015392
XAIX.DE     0.015345
DBA         0.015293
TQQQ        0.015268
PYPL        0.015163
BTC-USD     0.015158
DBO         0.015157
AMZN        0.015156
RENW.DE     0.015116
AAPL        0.015003
GLD         0.014832
VLUE        0.014755
EEM   

In [140]:
weights.to_csv("smaweights-tslareduced.csv")

in this case i want to create one portfolio with and one without tsla, bc it has 13% of portfolio which is too much

In [134]:
toInvest = weights * 10000
toInvest

ticker
TSLA        1352.349264
FAS          453.165855
URA          278.334852
NVDA         270.456395
TEAM         235.368574
AVAX-USD     210.310215
DBE          203.514563
DG           199.826177
LLY          191.570897
ETH-USD      190.824155
VAW          190.122874
NOC          182.155252
UPRO         180.774245
PGR          174.456870
WM           172.464681
GOOG         171.710369
VIS          165.257803
UNH          164.851212
AMD          162.694690
WOOD         161.352887
VPU          155.338138
KDP          153.067306
VDE          151.758066
VHT          150.573882
IWDA.AS      150.508657
VDC          145.916140
VFH          145.349952
META         145.325291
VTV          144.207676
L0CK.DE      143.369378
BTEC.L       142.691296
2B76.DE      142.076690
MSFT         140.052591
DBB          138.560063
VNQ          137.028228
FNDX         136.280104
XAIX.DE      135.815947
DBA          135.295200
TQQQ         135.044870
PYPL         133.996987
BTC-USD      133.943871
DBO      

In [135]:
bot = BaseBot("manual-simplesma-static-v1")
for ticker, money in toInvest.iteritems():
    bot.buy(ticker, money, amountInUSD=True)

next only assign 453$ to tsla, put the remaining 899 to all the other stoxx

In [136]:
toInvest["TSLA"] = 453.16
addPerStock = 899 / len(toInvest)
toInvest = toInvest + addPerStock
toInvest

ticker
TSLA        469.808148
FAS         469.814003
URA         294.983000
NVDA        287.104543
TEAM        252.016722
AVAX-USD    226.958363
DBE         220.162712
DG          216.474325
LLY         208.219045
ETH-USD     207.472303
VAW         206.771022
NOC         198.803400
UPRO        197.422393
PGR         191.105018
WM          189.112829
GOOG        188.358518
VIS         181.905951
UNH         181.499360
AMD         179.342839
WOOD        178.001035
VPU         171.986287
KDP         169.715454
VDE         168.406214
VHT         167.222030
IWDA.AS     167.156805
VDC         162.564288
VFH         161.998100
META        161.973439
VTV         160.855825
L0CK.DE     160.017527
BTEC.L      159.339445
2B76.DE     158.724838
MSFT        156.700740
DBB         155.208211
VNQ         153.676376
FNDX        152.928253
XAIX.DE     152.464095
DBA         151.943348
TQQQ        151.693018
PYPL        150.645135
BTC-USD     150.592019
DBO         150.586367
AMZN        150.574255
RENW

In [137]:
bot = BaseBot("manual-simplesma-static-reducedtsla-v1")
for ticker, money in toInvest.iteritems():
    bot.buy(ticker, money, amountInUSD=True)